## simple notebook to show basic steps for preparing era5 data for use

In many instances, it is possible to identify the single closest ERA5 point to our data, and use it to build rough models. Monthly wind speed aggregate values are typically used with great success with monthly wind plant aggregate values of gross energy (daily average or adjusted for number of days in month) as the gross production will/should follow the resource wind speed, so a linear model with wind speed as the predictor and gross energy as the predictand can be quite strong and reliable. It is not the absolute value of the wind speed that matters, but the pattern of change, month to month.

Small improvements might be made by interpolating the 4 surrounding points to the site coordinates to approximate the relative impact of the 4 surrounding sites. It is common for the interpolated wind speed value to have a slightly higher correlation and rsquared than the closest point.

In cases where models will be built at higher resolution, additional adjustments may reduce model uncertainty.  Besides interpolation to the site coordinates, we can consider the elevation of teh measurements.   ERA5 has u and v vector measurements of easting and northing at both 10 and 100m as well as higher elevations based on pressure levels, along with temperature at 2m and pressure at ground level.  The u and v parameters can be converted to wind speed and direction,  we can adjust speed based on shear from the 10/100 m measurements to our hub height (an average of 75 m at Kelmarsh) and there are lapse rates - changes with elevation for pressure and temperature.

in the simplified code that follows, we'll employ all the more common adjustments to demonstrate what is possible.  Again, it is part of model design, development, testing and verification to confirm that the additional steps are adding value to the modeling process by reducing model uncertainty. As many of these adjustments are numeric transformations, most models will self adjust to the raw unadjusted parameter, but it should be confirmed, not assumed.



### Key libaries

* python version: 3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]
* numpy version: 1.26.2
* polars version: 1.17.1

In [1]:
print('python version:', __import__('sys').version)

import math # to use math functions like radians, atan2
from calendar import monthrange # to get the number of days in a month
import time # for time.sleep() to wait for the API to respond
from pathlib import Path # to work with file paths
cwd = Path.cwd()

import numpy as np
print('numpy version:', np.__version__)
import polars as pl # to use polars dataframes for our data
import polars.selectors as cs # to use column selectors in polars
# Set the display width
pl.Config.set_tbl_cols(100)  # Set the number of polars df columns to display when printing
pl.Config.set_tbl_width_chars(200)  # Set the width of polars df columns in characters when printing
print('polars version:', pl.__version__)


python version: 3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]
numpy version: 1.26.2
polars version: 1.17.1


### Get the data

Our previous notebook saved the data to a file. 

That is a great habit to get into, breaking a long series of steps into pieces and saving work at the end of each step.

In [2]:
# get the data saved to the output folder
era5_data = pl.read_parquet(cwd / 'era5_data' / 'era5_data.parquet')
era5_data.describe()

statistic,TimeStamp_UTC,latitude,longitude,EnvTmp_Alt2m,HorWdU_Alt10m,HorWdV_Alt10m,EnvPres_Alt0m,HorWdU_Alt100m,HorWdV_Alt100m
str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""35136""",35136.0,35136.0,35136.0,35136.0,35136.0,35136.0,35136.0,35136.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""2020-07-01 23:30:00""",52.375,-0.875,283.960938,1.658201,1.06936,100122.273438,2.7473,1.635691
"""std""",null,0.125002,0.125002,5.814668,3.47585,3.56183,1190.049805,5.531496,5.616271
"""min""","""2020-01-01 00:00:00""",52.25,-1.0,270.56134,-7.789871,-9.979416,95628.28125,-12.607285,-14.742493
"""25%""","""2020-04-01 12:00:00""",52.25,-1.0,279.648254,-1.023514,-1.47049,99399.414062,-1.620895,-2.700119
"""50%""","""2020-07-02 00:00:00""",52.5,-0.75,283.434387,1.922028,1.003311,100266.976562,3.468735,1.528824
"""75%""","""2020-10-01 11:00:00""",52.5,-0.75,287.909241,4.120361,3.574753,100931.398438,6.831512,5.836319
"""max""","""2020-12-31 23:00:00""",52.5,-0.75,307.781555,12.20639,14.168121,103669.453125,18.772491,21.104904


### adjust parameters to avg kelmarsh hub height 75 m

we jump to speed and direction at 10 and 100 so we can use a shear calc to get 75m wind speed, and we extrapolate wind direction, then we convert these back to u and v at 75 m


In [3]:

# Constants  used in functions

R = 287.05  # Specific gas constant for dry air (J/(kg·K))
g = 9.80665  # Acceleration due to gravity (m/s²)
L = -0.0065  # Standard temperature lapse rate (K/m)
kelmarsh_avg_hh = 75  # Height difference (m)
z0 = 10  # Reference height (m)
z1 = 100  # Second height (m)

# functions

def calculate_temperature(df, base_temp_col, height_diff, alias):
    """
    Calculate the temperature at a different height based on the lapse rate.

    Parameters:
    df (pl.DataFrame): The input DataFrame.
    base_temp_col (str): The column name for the base temperature.
    height_diff (float): The height difference in meters.
    alias (str): The alias for the new temperature column.

    Returns:
    pl.DataFrame: DataFrame with the new temperature column.
    """
    return df.with_columns(
        (pl.col(base_temp_col) + L * height_diff).alias(alias)
    )

def calculate_pressure(df, base_pres_col, temp_col, height_diff, alias):
    """
    Calculate the pressure at a different height based on the barometric formula.

    Parameters:
    df (pl.DataFrame): The input DataFrame.
    base_pres_col (str): The column name for the base pressure.
    temp_col (str): The column name for the temperature.
    height_diff (float): The height difference in meters.
    alias (str): The alias for the new pressure column.

    Returns:
    pl.DataFrame: DataFrame with the new pressure column.
    """
    return df.with_columns(
        (pl.col(base_pres_col) * np.exp(-g * height_diff / (R * pl.col(temp_col)))).alias(alias)
    )

def calculate_air_density(df, pres_col, temp_col, alias):
    """
    Calculate the air density based on the ideal gas law.

    Parameters:
    df (pl.DataFrame): The input DataFrame.
    pres_col (str): The column name for the pressure.
    temp_col (str): The column name for the temperature.
    alias (str): The alias for the new air density column.

    Returns:
    pl.DataFrame: DataFrame with the new air density column.
    """
    return df.with_columns(
        (pl.col(pres_col) / (R * pl.col(temp_col))).alias(alias)
    )

def wswd_from_uv(df, u_col, v_col, speed_alias, dir_alias):
    """
    Calculate wind speed and direction from u and v wind components.

    Parameters:
    df (pl.DataFrame): The input DataFrame.
    u_col (str): The column name for the u wind component.
    v_col (str): The column name for the v wind component.
    speed_alias (str): The alias for the wind speed column.
    dir_alias (str): The alias for the wind direction column.

    Returns:
    pl.DataFrame: DataFrame with the new wind speed and direction columns.
    """
    return df.with_columns(
        (pl.col(u_col)**2 + pl.col(v_col)**2).sqrt().alias(speed_alias),
        (pl.arctan2(pl.col(v_col), pl.col(u_col)) * pl.lit(180 / np.pi)).alias(dir_alias)
    )

def uv_from_wswd(df, speed_col, dir_col, u_alias, v_alias):
    """
    Calculate u and v wind components from wind speed and direction.

    Parameters:
    df (pl.DataFrame): The input DataFrame.
    speed_col (str): The column name for the wind speed.
    dir_col (str): The column name for the wind direction.
    u_alias (str): The alias for the u wind component column.
    v_alias (str): The alias for the v wind component column.

    Returns:
    pl.DataFrame: DataFrame with the new u and v wind component columns.
    """
    df = df.with_columns(
        (pl.col(dir_col) * pl.lit(np.pi / 180)).alias(f"{dir_col}_rad")
    )
    return df.with_columns(
        (-pl.col(speed_col) * pl.col(f"{dir_col}_rad").sin()).alias(u_alias),
        (-pl.col(speed_col) * pl.col(f"{dir_col}_rad").cos()).alias(v_alias)
    ).drop([f"{dir_col}_rad"])

def interpolate_wind_direction(df, dir_col_Alt10m, dir_col_Alt100m, height_diff, alias):
    """
    Interpolate wind direction between two heights.

    Parameters:
    df (pl.DataFrame): The input DataFrame.
    dir_col_Alt10m (str): The column name for the wind direction at 10 meters.
    dir_col_Alt100m (str): The column name for the wind direction at 100 meters.
    height_diff (float): The height difference in meters.
    alias (str): The alias for the new interpolated wind direction column.

    Returns:
    pl.DataFrame: DataFrame with the new interpolated wind direction column.
    """
    return df.with_columns(
        ((pl.col(dir_col_Alt10m) + (pl.col(dir_col_Alt100m) - pl.col(dir_col_Alt10m)) * height_diff / (z1 - z0))).alias(alias)
    )

# new function - determine offset of the offset from UTC for a given longitude
def offset_from_longitude(longitude):
    """
    Determine the offset from UTC for a given longitude.

    Parameters:
    longitude (float): The longitude in degrees.

    Returns:
    int: The offset from UTC in hours.
    """
    return round(longitude / 15)

    
# Assuming era5_data is already created and contains the necessary columns
# era5_data = ... (created in the previous cell)

# Calculate temperature at 75 meters
era5_data = calculate_temperature(era5_data, "EnvTmp_Alt2m", kelmarsh_avg_hh - 2, "EnvTmp_Alt75m")

# Calculate pressure at 75 meters
era5_data = calculate_pressure(era5_data, "EnvPres_Alt0m", "EnvTmp_Alt75m", kelmarsh_avg_hh, "EnvPres_Alt75m")

# Calculate air density at the surface (0 meters)
era5_data = calculate_air_density(era5_data, "EnvPres_Alt0m", "EnvTmp_Alt2m", "AirDen_Alt0m")

# Calculate air density at 75 meters
era5_data = calculate_air_density(era5_data, "EnvPres_Alt75m", "EnvTmp_Alt75m", "AirDen_Alt75m")

# Calculate temperature at 100 meters
era5_data = calculate_temperature(era5_data, "EnvTmp_Alt2m", 100 - 2, "EnvTmp_Alt100m")

# Calculate pressure at 100 meters
era5_data = calculate_pressure(era5_data, "EnvPres_Alt0m", "EnvTmp_Alt100m", 100, "EnvPres_Alt100m")

# Calculate air density at 100 meters
era5_data = calculate_air_density(era5_data, "EnvPres_Alt100m", "EnvTmp_Alt100m", "AirDen_Alt100m")

# Calculate wind speed and direction at 10 meters and 100 meters
era5_data = wswd_from_uv(era5_data, "HorWdU_Alt10m", "HorWdV_Alt10m", "HorWdSpd_Alt10m", "HorWdDir_Alt10m")
era5_data = wswd_from_uv(era5_data, "HorWdU_Alt100m", "HorWdV_Alt100m", "HorWdSpd_Alt100m", "HorWdDir_Alt100m")

# Calculate wind shear exponent using wind speeds at 10 meters and 100 meters
era5_data = era5_data.with_columns(
    (pl.col("HorWdSpd_Alt100m") / pl.col("HorWdSpd_Alt10m")).log().alias("log_wind_speed_ratio"),
    (pl.lit(z1 / z0).log()).alias("log_height_ratio")
)

era5_data = era5_data.with_columns(
    (pl.col("log_wind_speed_ratio") / pl.col("log_height_ratio")).alias("wind_shear_exponent")
)

# Calculate wind speed at 75 meters using the power law
era5_data = era5_data.with_columns(
    (pl.col("HorWdSpd_Alt10m") * (kelmarsh_avg_hh / z0)**pl.col("wind_shear_exponent")).alias("HorWdSpd_Alt75m")
)

# Interpolate wind direction at 75 meters
era5_data = interpolate_wind_direction(era5_data, "HorWdDir_Alt10m", "HorWdDir_Alt100m", kelmarsh_avg_hh - z0, "HorWdDir_Alt75m")

# Calculate U and V components at 75 meters
era5_data = uv_from_wswd(era5_data, "HorWdSpd_Alt75m", "HorWdDir_Alt75m", "HorWdU_Alt75m", "HorWdV_Alt75m")

# Drop intermediate columns
era5_data = era5_data.drop(["log_wind_speed_ratio", "log_height_ratio", "wind_shear_exponent"])

# Reorder columns
columns_order = [
    "TimeStamp_UTC", "latitude", "longitude",
    "HorWdU_Alt10m", "HorWdV_Alt10m", "HorWdSpd_Alt10m", "HorWdDir_Alt10m", "EnvPres_Alt0m", "EnvTmp_Alt2m", "AirDen_Alt0m",
    "HorWdU_Alt75m", "HorWdV_Alt75m", "HorWdSpd_Alt75m", "HorWdDir_Alt75m", "EnvPres_Alt75m", "EnvTmp_Alt75m", "AirDen_Alt75m",
    "HorWdU_Alt100m", "HorWdV_Alt100m", "HorWdSpd_Alt100m", "HorWdDir_Alt100m", "EnvPres_Alt100m", "EnvTmp_Alt100m", "AirDen_Alt100m"
]
era5_data = era5_data.select(columns_order)

# Print the DataFrame with all calculations
print(era5_data.head())

shape: (5, 24)
┌────────┬────────┬────────┬────────┬────────┬────────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ TimeSt ┆ latitu ┆ longit ┆ HorWdU ┆ HorWdV ┆ HorWdS ┆ HorWdD ┆ EnvPr ┆ EnvTm ┆ AirDe ┆ HorWd ┆ HorWd ┆ HorWd ┆ HorWd ┆ EnvPr ┆ EnvTm ┆ AirDe ┆ HorWd ┆ HorWd ┆ HorWd ┆ HorWd ┆ EnvPr ┆ EnvTm ┆ AirDe │
│ amp_UT ┆ de     ┆ ude    ┆ _Alt10 ┆ _Alt10 ┆ pd_Alt ┆ ir_Alt ┆ es_Al ┆ p_Alt ┆ n_Alt ┆ U_Alt ┆ V_Alt ┆ Spd_A ┆ Dir_A ┆ es_Al ┆ p_Alt ┆ n_Alt ┆ U_Alt ┆ V_Alt ┆ Spd_A ┆ Dir_A ┆ es_Al ┆ p_Alt ┆ n_Alt │
│ C      ┆ ---    ┆ ---    ┆ m      ┆ m      ┆ 10m    ┆ 10m    ┆ t0m   ┆ 2m    ┆ 0m    ┆ 75m   ┆ 75m   ┆ lt75m ┆ lt75m ┆ t75m  ┆ 75m   ┆ 75m   ┆ 100m  ┆ 100m  ┆ lt100 ┆ lt100 ┆ t100m ┆ 100m  ┆ 100m  │
│ ---    ┆ f32    ┆ f32    ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ m     ┆ m     ┆ --- 

In [4]:
for i,c in enumerate(era5_data.columns):
    print(c, end = ', ')
    if i % 8 == 0:
        print()

TimeStamp_UTC, 
latitude, longitude, HorWdU_Alt10m, HorWdV_Alt10m, HorWdSpd_Alt10m, HorWdDir_Alt10m, EnvPres_Alt0m, EnvTmp_Alt2m, 
AirDen_Alt0m, HorWdU_Alt75m, HorWdV_Alt75m, HorWdSpd_Alt75m, HorWdDir_Alt75m, EnvPres_Alt75m, EnvTmp_Alt75m, AirDen_Alt75m, 
HorWdU_Alt100m, HorWdV_Alt100m, HorWdSpd_Alt100m, HorWdDir_Alt100m, EnvPres_Alt100m, EnvTmp_Alt100m, AirDen_Alt100m, 

### interpolate values to the site location

era5_data now has a lot of data at the 4 coordinates surrounding the site - we could not create an interpolated dataset - so the weighted influence of hte surrounding points can better related to site conditions






In [5]:

def get_surrounding_grid_points(lat, lon, interval=0.25):
    # Calculate the nearest grid point
    nearest_lat = round(lat / interval) * interval
    nearest_lon = round(lon / interval) * interval

    # Calculate surrounding grid points
    lat_points = [nearest_lat - interval, nearest_lat, nearest_lat + interval]
    lon_points = [nearest_lon - interval, nearest_lon, nearest_lon + interval]

    # Generate all combinations of surrounding grid points
    surrounding_points = [(lat, lon) for lat in lat_points for lon in lon_points]
    
    return surrounding_points

def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Compute differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

def get_closest_grid_points(lat, lon, num_points=4):
    surrounding_points = get_surrounding_grid_points(lat, lon)
    distances = [(point, haversine(lat, lon, point[0], point[1])) for point in surrounding_points]
    distances.sort(key=lambda x: x[1])
    closest_points = [point for point, distance in distances[:num_points]]
    return closest_points

# reminder where we are - Kelmarsh site:
lat = 52.40
lon = -0.943
closest_points = get_closest_grid_points(lat, lon) # function from up above

# Print the closest points and their distances
distance_list = []
for point in closest_points:
    distance = np.round(haversine(lat, lon, point[0], point[1]), 3)
    print(f'For point {point}, distance is {distance} km from Kelmarsh at {lat}, {lon}')
    # create a new polars dataframe with lat/long/distance as columns to use in next step
    point_lat = point[0]
    point_lon = point[1]
    tmp  = pl.DataFrame({
        'latitude': [point_lat],
        'longitude': [point_lon],
        'distance': [distance] })
    distance_list.append(tmp)
distances_df = (pl.concat(distance_list)).sort(['latitude','longitude'])

distances_df = distances_df.with_columns(  (pl.lit('{') + pl.col('latitude').cast(pl.Utf8) + pl.lit(',') + pl.col('longitude').cast(pl.Utf8) + pl.lit('}')).alias('pivot_col_name')) 
print(distances_df)

# consider variables used and if it is reasonable to store as float32 instead of float64 as it takes up half the space in RAM and you can deal with larger datasets
distances_df = distances_df.cast({cs.float():pl.Float32})

era5_data = distances_df.join(era5_data, on=['latitude', 'longitude'], how='inner')    

For point (52.5, -1.0), distance is 11.771 km from Kelmarsh at 52.4, -0.943
For point (52.25, -1.0), distance is 17.123 km from Kelmarsh at 52.4, -0.943
For point (52.5, -0.75), distance is 17.167 km from Kelmarsh at 52.4, -0.943
For point (52.25, -0.75), distance is 21.219 km from Kelmarsh at 52.4, -0.943
shape: (4, 4)
┌──────────┬───────────┬──────────┬────────────────┐
│ latitude ┆ longitude ┆ distance ┆ pivot_col_name │
│ ---      ┆ ---       ┆ ---      ┆ ---            │
│ f64      ┆ f64       ┆ f64      ┆ str            │
╞══════════╪═══════════╪══════════╪════════════════╡
│ 52.25    ┆ -1.0      ┆ 17.123   ┆ {52.25,-1.0}   │
│ 52.25    ┆ -0.75     ┆ 21.219   ┆ {52.25,-0.75}  │
│ 52.5     ┆ -1.0      ┆ 11.771   ┆ {52.5,-1.0}    │
│ 52.5     ┆ -0.75     ┆ 17.167   ┆ {52.5,-0.75}   │
└──────────┴───────────┴──────────┴────────────────┘


In [6]:
import polars as pl
import numpy as np

# Example usage
lat = 52.40
lon = -0.943

# Assuming era5_data and distances_df are already created and contain the necessary columns
# era5_data contains columns: TimeStamp_UTC, latitude, longitude, distance, pivot_col_name and other data columns
# distances_df contains columns: latitude, longitude, distance and pivot_col_name

# Perform inverse distance weighting interpolation
def idw_interpolation(distances, values):
    weights = 1 / distances
    weights /= weights.sum()  # Normalize weights
    interpolated_value = np.dot(weights, values)
    return interpolated_value

def interpolate_columns(era5_data, distances_df, lat, lon):
    interpolated_series = []

    for col in era5_data.columns:
        if col not in ["latitude", "longitude", "TimeStamp_UTC", "distance", 'pivot_col_name']:
            # Pivot the DataFrame on latitude and longitude combinations
            tmp = (era5_data.select('TimeStamp_UTC', 'pivot_col_name', col)
                           .pivot(index='TimeStamp_UTC', on=['pivot_col_name'], values=col))

            # Extract the pivot column names from distances_df
            pivot_col_names = distances_df['pivot_col_name'].to_list()

            # Extract the values for the surrounding points from the pivoted DataFrame
            values = tmp.select(pivot_col_names).to_numpy()

            # Extract the distances for the surrounding points from distances_df
            distances = distances_df['distance'].to_numpy()

            # Ensure that the values array is correctly shaped
            values = values.reshape(-1, len(distances))

            # Apply IDW interpolation for each row in the pivoted DataFrame
            interpolated_values = np.apply_along_axis(
                lambda row: idw_interpolation(distances, row),
                axis=1,
                arr=values
            )

            # Create a new DataFrame with the interpolated values
            interpolated_df = tmp.select("TimeStamp_UTC").with_columns([
                pl.Series(name=f"{lat},{lon}", values=interpolated_values)
            ])

            # Join the interpolated values back to the tmp DataFrame
            result_df = tmp.join(interpolated_df, on="TimeStamp_UTC")

            # Unpivot the DataFrame back to long format
            long_df = result_df.unpivot(
                index="TimeStamp_UTC",
                variable_name="location",
                value_name=col
            )

            # Split the 'location' column into 'latitude' and 'longitude'
            long_df = long_df.with_columns([
                pl.col("location").str.split_exact(",", 1).alias("split_location")
            ]).with_columns([
                pl.col("split_location").struct[0].str.strip_chars("{}").cast(pl.Float64).alias("latitude"),
                pl.col("split_location").struct[1].str.strip_chars("{}").cast(pl.Float64).alias("longitude")
            ]).drop("split_location", "location")
            
            interpolated_series.append(long_df)


    # the dfs in interpolated_series have to be joined on TimeStamp_UTC, latitude, longitude, not concatenated
    era5_data_interp = interpolated_series[0]
    for i in range(1, len(interpolated_series)):
        era5_data_interp = era5_data_interp.join(interpolated_series[i], on=['TimeStamp_UTC', 'latitude', 'longitude'], how='inner')   
        
    

    return era5_data_interp

# Interpolate all columns
era5_data_interp = interpolate_columns(era5_data, distances_df, lat, lon)

# Print the resulting DataFrame
print(era5_data_interp.head())

shape: (43_920, 24)
┌────────┬────────┬────────┬────────┬────────┬────────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ TimeSt ┆ HorWdU ┆ latitu ┆ longit ┆ HorWdV ┆ HorWdS ┆ HorWdD ┆ EnvPr ┆ EnvTm ┆ AirDe ┆ HorWd ┆ HorWd ┆ HorWd ┆ HorWd ┆ EnvPr ┆ EnvTm ┆ AirDe ┆ HorWd ┆ HorWd ┆ HorWd ┆ HorWd ┆ EnvPr ┆ EnvTm ┆ AirDe │
│ amp_UT ┆ _Alt10 ┆ de     ┆ ude    ┆ _Alt10 ┆ pd_Alt ┆ ir_Alt ┆ es_Al ┆ p_Alt ┆ n_Alt ┆ U_Alt ┆ V_Alt ┆ Spd_A ┆ Dir_A ┆ es_Al ┆ p_Alt ┆ n_Alt ┆ U_Alt ┆ V_Alt ┆ Spd_A ┆ Dir_A ┆ es_Al ┆ p_Alt ┆ n_Alt │
│ C      ┆ m      ┆ ---    ┆ ---    ┆ m      ┆ 10m    ┆ 10m    ┆ t0m   ┆ 2m    ┆ 0m    ┆ 75m   ┆ 75m   ┆ lt75m ┆ lt75m ┆ t75m  ┆ 75m   ┆ 75m   ┆ 100m  ┆ 100m  ┆ lt100 ┆ lt100 ┆ t100m ┆ 100m  ┆ 100m  │
│ ---    ┆ ---    ┆ f64    ┆ f64    ┆ ---    ┆ ---    ┆ ---    ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ m     ┆ m     ┆